# D1 — Detecção de Anomalias em Telemetria (Médio)

Detecte anomalias em telemetria mista de robôs. Retorne escores de anomalia por linha.

### Contexto
Robôs transmitem telemetria de motores, IMUs, sensores térmicos e nós de computação. Falhas se manifestam como desvios sutis que devem ser sinalizados antes de ultrapassarem limites de segurança. Simulamos telemetria mista via [`common.data_utils.telemetry_anomaly_dataset`](common/data_utils.py): normais seguem uma Gaussiana de média zero, enquanto anomalias adicionam saliências correlacionadas entre canais para emular travamentos de atuadores ou eventos de superaquecimento.

## Especificação de Entrada e Saída
**Entrada:** `X (N×d)` telemetria  
**Saída:** `scores (N,)` escores de anomalia (maior = mais anômalo)

Orientações de runtime/recursos: qubits ≤ 10, passos do otimizador ≤ 150; mantenha a inferência por lote abaixo de ~0,05 s.

### Dicas de solução
- Ajuste um modelo nominal usando apenas inliers (ex.: QSVM de uma classe com [`common.quantum_utils.feature_map`](common/quantum_utils.py)) e pontue pela distância à fronteira aprendida.
- Caminho de autoencoder quântico: codifique vetores de telemetria em um latente de baixa dimensão com um circuito variacional; o erro de reconstrução vira o escore de anomalia.
- Calibre limiares de decisão em telemetria limpa de validação para que escores altos correspondam a alertas acionáveis.

_Orientação de baseline:_ [`common.baselines.baseline_anomaly_threshold`](common/baselines.py) atinge AUROC ≈ 0.92–0.94 em seeds variadas; testes ocultos requerem AUROC ≥ 0.90.

## Setup

In [ ]:
import numpy as np
from common import data_utils as du
from common import baselines as bl
from common import quantum_utils as qu
np.random.seed(1337)


## Baseline (referência)

In [ ]:
X, y = du.telemetry_anomaly_dataset(n=600, d=8, seed=0, anomaly_rate=0.03)
scores = bl.baseline_anomaly_threshold(X)
from sklearn.metrics import roc_auc_score
au = roc_auc_score(y, scores)
print("AUROC do baseline:", au)


## Sua tarefa: implementar `solve(...)`

In [ ]:
def solve(X):
    # Substitua por kernel de uma classe quântico, autoencoder quântico ou detector híbrido que aumente o AUROC.
    return bl.baseline_anomaly_threshold(X)


## Testes públicos (rápidos)

In [ ]:
from sklearn.metrics import roc_auc_score
X, y = du.telemetry_anomaly_dataset(n=800, d=8, seed=1, anomaly_rate=0.03)
scores = solve(X)
au = roc_auc_score(y, scores)
print("AUROC público:", au)
assert scores.shape == (len(y),)
assert au >= 0.90, "AUROC público abaixo do limiar"
print("OK")


> Testes adicionais serão executados pelos organizadores com seeds/tamanhos diferentes.